# Храмов А.А.
# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from math import sqrt
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
#%load_ext pycodestyle_magic

In [1589]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype(np.float)
        r_s = r_s.astype(np.float)
        return l_s * (1 - np.sum((l_c / l_s) ** 2, axis=1)[:, np.newaxis]) + r_s * (
                1 - np.sum((r_c / r_s) ** 2, axis=1)[:, np.newaxis])

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(l_c * np.log1p(l_c / l_s) + r_c * np.log1p(r_c / r_s), axis=1).reshape(-1, 1) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c, axis=1).reshape(-1, 1) + np.max(r_c, axis=1).reshape(-1, 1)) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # возвращает отсортированный х и соответствующие ему у
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # количество уникальных элементов в у, соответственно кол-во классов в ноде
        # class_number = np.unique(y).shape[0]
        class_number = self.num_class
        cut = self.min_samples_split / 2 - 1
        # print class_number == self.num_class
        # Что делает этот блок кода?
        # min_samples_split - минимальное количество семплов для разделения ноды  
        # splitted_sorted_y - массив с отсортированными у от которого взяли середину
        # (без min_samples_split слева и справа)
        # если элементов меньше или равно, чем min_samples_split, то он вернет пустой массив
        if cut == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut:-cut]

        # r_border_ids - это массив с индексами элементов из splitted_sorted_y 
        # которые отличаются от предыдущего элемента
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (cut + 1)

        # выходим из функции и возвращаем порог = бесконечности
        if len(r_border_ids) == 0:
            # print "inf======"
            return float('+inf'), None

        # Cтавит единицы там, где индексы идут подряд 
        eq_el_count = r_border_ids - np.append([cut], r_border_ids[:-1])
        # создает матрицу размером NхМ, N - сколько раз целевая переменная меняется на х_sorted
        #                               M - кол-во различных значений которые принимает целевая переменная в y
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        # Cтавим 1 в каждой строчке на место, которое соответствует значению целевой переменной, 
        # которе у нее было до смены этого значения
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1

        # В этой матрице на тех местах, где в one_hot_code стоят единицы,
        # стоит количество одинаковых, подряд идущих элементов между сменой целевой переменной
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)

        # В первую строчку в места соответствующих значений целевой переменной 
        # прибавляем их количество из отрезанного ранее куска 
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:cut],
                                                                minlength=class_number)

        # Считаем сколько семплов попадут в правый и в левый потомок
        # при этом строчки отвечают за индекс разбиения (через r_border_ids)
        # а столбцы за значение целевой переменной
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y, minlength=class_number) - l_class_count
        # количество семплов в левом и правом потомке для каждого разбиения
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # находим impurity для всех разбиений
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)

        # получаем индекс разбиения с минимальным impurity
        idx = np.argmin(gs)

        # получаем индекс последнего (крайнего) элемента, который удовлетиворяет
        # выбранному разбиению
        left_el_id = l_sizes[idx][0]
        # возвращаем impurity и threshold
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):

        (classes, counts) = np.unique(y, return_counts=True)
        cl_ind = np.argmax(counts)
        cl = classes[cl_ind]
        if depth == self.max_depth or counts[cl_ind] / y.size >= \
                self.sufficient_share:
            self.tree[node_id] = (self.LEAF_TYPE, cl)
            return
        feature_ids = self.get_feature_ids(x.shape[1])
        imp = np.zeros(len(feature_ids))
        threshold = np.zeros(len(feature_ids))
        for i in feature_ids:
            imp[i], threshold[i] = self.__find_threshold(x[:, i], y)

        imp[pred_f] = np.inf

        feature = np.argmin(imp)
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature, threshold[feature])

        if y_l.size == 0:
            nums, counts = np.unique(y_r, return_counts=True)
            cl = nums[np.argmax(counts)]
            self.tree[node_id] = (self.LEAF_TYPE, cl)
            return
        if y_r.size == 0:
            nums, counts = np.unique(y_l, return_counts=True)
            self.tree[node_id] = (self.LEAF_TYPE, nums[np.argmax(counts)])
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, feature, threshold[feature])
        l_coef = y_l.size / y.size
        r_coef = y_r.size / y.size
        self.feature_importances_[feature] += self.gain(y) - l_coef * self.gain(y_l) - r_coef * self.gain(y_r)
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, feature)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, feature)

    def gain(self, y):
        _, counts = np.unique(y, return_counts=True)

        return 1 - np.sum((counts / y.size) ** 2)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                if self.tree.get(2 * node_id + 1, -1) != -1:
                    return self.__predict_class(x, 2 * node_id + 1)
                else:
                    return self.max_frec
            else:
                if self.tree.get(2 * node_id + 2, -1) != -1:
                    return self.__predict_class(x, 2 * node_id + 2)
                else:
                    return self.max_frec
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    def score(self, x, y_test):
        return f1_score(y_pred=my_clf.predict(x), y_true=y_test, average='macro')

    def feature(self):
        return self.feature_importances_


In [1590]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [1591]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)
X_train.shape

(160, 13)

## Проверка скорости работы на wine

In [1592]:
from time import time

In [1593]:
%time clf.fit(X_train, y_train)


CPU times: user 3.6 ms, sys: 98 µs, total: 3.7 ms
Wall time: 77.9 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [1594]:
%time my_clf.fit(X_train, y_train)

CPU times: user 10.6 ms, sys: 70 µs, total: 10.7 ms
Wall time: 10.3 ms


## Проверка качества работы на wine

In [1595]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

In [1596]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9474747474747475

## Подготовка данных Speed Dating Data 

In [1579]:
import pandas as pd
import numpy as np

df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv')
df = df.iloc[:, :97]
df.iid.nunique()

df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df.drop_duplicates(subset=['iid']).gender.value_counts()
df.drop_duplicates(subset=['iid']).condtn.value_counts()
df = df.drop(['condtn'], axis=1)

df = df.drop(['round'], axis=1)
df = df.drop(['position', 'positin1'], axis=1)
df = df.drop(['order'], axis=1)
df = df.drop(['partner'], axis=1)
df = df.drop(['age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
             axis=1)
df.drop_duplicates(subset=['iid']).age
df.drop_duplicates('iid').age.isnull().sum()
df = df.dropna(subset=['age'])

df.field_cd.isnull().sum()

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)

df = df.drop(['field'], axis=1)
df = df.drop(['undergra'], axis=1)

df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)

df.drop_duplicates('iid').mn_sat
df.drop_duplicates('iid').mn_sat.isnull().sum()
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)

df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.drop_duplicates('iid').tuition
df.drop_duplicates('iid').tuition.isnull().sum()
df.loc[:, 'tuition'] = df.tuition.fillna(-999)

df.drop_duplicates('iid').race.value_counts()
df.drop_duplicates('iid').age.isnull().sum()
df.drop_duplicates('iid').race
df.drop_duplicates('iid').imprace.isnull().sum()
df.drop_duplicates('iid').imprelig.isnull().sum()

df = df.dropna(subset=['imprelig', 'imprace'])

df = df.drop(['from', 'zipcode'], axis=1)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)

df.drop_duplicates('iid').loc[:, 'income']

df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)

df = df.dropna(subset=['date'])

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)

df = df.drop(['career'], axis=1)
df.loc[:, ['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
           'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga']
].isnull().sum()

df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
              'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
df.drop_duplicates('iid').exphappy.isnull().sum()
df.drop_duplicates('iid').expnum.isnull().sum()
df = df.drop(['expnum'], axis=1)

feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)
temp.loc[idx,]
idx = ((temp.wave >= 6) & (temp.wave <= 9))

temp.loc[idx,]
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']

temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 90) & (temp.totalsum != 0)
temp.loc[idx,]

idx = ((temp.wave >= 6) & (temp.wave <= 9))
temp.loc[idx,]

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:,
                                                                                      'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)

df = df.drop(['wave'], axis=1)
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
    .drop(['gender'], axis=1) \
    .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
    .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
    .dropna()
df_female.columns = df_female.columns + '_f'

df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
df_pair.head()

X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values


In [1580]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train.shape

(3599, 63)

## Проверка скорости работы на Speed Dating Data 

In [1581]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)

clf = DecisionTreeClassifier(min_samples_split=2)

In [1582]:
%time clf.fit(X_train, y_train)

CPU times: user 68.5 ms, sys: 9 µs, total: 68.5 ms
Wall time: 68 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [1583]:
%time my_clf.fit(X_train, y_train)

CPU times: user 937 ms, sys: 0 ns, total: 937 ms
Wall time: 936 ms


## Проверка качества работы на Speed Dating Data

In [1584]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro') 

0.5306666666666666

In [1585]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro') 

0.55962781447546

## Задание 3

## Задание 4

In [1586]:
features_names = df_pair.columns[1:]

In [1587]:
print "sklern"
a=[]
imp=clf.feature_importances_[np.argsort(clf.feature_importances_)][-10:]
fe=features_names[np.argsort(clf.feature_importances_)][-10:]
for i in range(10):
    print [imp[i],fe[i]]


sklern
[0.023148909886176374, 'imprace']
[0.023222541109045428, 'intel1_1_f']
[0.023375998199870126, 'field_cd_f']
[0.025016617654146846, 'sinc3_1']
[0.0273507246216168, 'fun1_1_f']
[0.02943430218237748, 'fun2_1_f']
[0.0300530191182279, 'sinc1_1_f']
[0.033247065532509085, 'exphappy']
[0.03334269590358133, 'income_f']
[0.092457067409491, 'int_corr']


In [1588]:
print "my Tree"
a=[]
imp=my_clf.feature()[np.argsort(my_clf.feature())][-10:]
fe=features_names[np.argsort(my_clf.feature())][-10:]
for i in range(10):
    print [imp[i],fe[i]]

my Tree
[0.0011114198388441233, 'age_f']
[0.0011114198388441233, 'imprace_f']
[0.0011114198388441233, 'race']
[0.0011114198388441233, 'age']
[0.0013892747985551543, 'amb1_1_f']
[0.0013892747985551543, 'amb1_1']
[0.0016671297582661851, 'field_cd']
[0.0016671297582661851, 'income_f']
[0.0016671297582661851, 'samerace']
[0.004167824395665463, 'int_corr']


## Задание 5

In [1230]:
clf = RandomForestClassifier()
param_dist = {"max_depth": range(5,15),
              "min_samples_split": range(5,15),
              "max_features": ["sqrt", None],
              "criterion": ["gini", "entropy"]}

greed_search = GridSearchCV(clf, param_grid=param_dist)
greed_search.fit(X_train,y_train)
print greed_search.best_params_

{'max_features': 'sqrt', 'min_samples_split': 9, 'criterion': 'gini', 'max_depth': 10}
